In [1]:
#from scipy import fft
from scipy import signal as sig
from scipy.fft import fft, fftfreq
from IPython.display import Audio
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math
import wave
import numpy as np
import matplotlib.pyplot as plt
import sys
import os
import librosa, librosa.display
import IPython.display as ipd
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import scipy.io.wavfile as wavfile
import scipy
import scipy.fftpack
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
import warnings
warnings.simplefilter('ignore')

In [2]:
# Testing Files present at Files\cough_wavs_sample

In [3]:
def custom_fft(y, fs):
    T = 1.0 / fs
    N = y.shape[0]
    yf = fft(y)
    xf = np.linspace(0.0, 1.0/(2.0*T), N//2)
    vals = 2.0/N * np.abs(yf[0:N//2])  
    return xf, vals

In [4]:
def plot_wave(file):
    demo_audio = file #shorter clip
    demo_clip, demo_sample_rate = librosa.load(demo_audio, sr=None)
    demo_xf, demo_vals = custom_fft(demo_clip, demo_sample_rate)

    trace = [go.Scatter(
    x=np.linspace(0, demo_sample_rate/len(demo_clip), len(demo_clip)), 
    y=demo_clip
    )]
    layout = go.Layout(
        title = 'Waveform <br><sup>Interactive</sup>',
        yaxis = dict(title='Amplitude'),
        xaxis = dict(title='Time'),
        )
    fig = go.Figure(data=trace, layout=layout)
    fig.show()
    
file = r'../input/covid-cough-wavs/cleaned_data/Positive/1084_Positive_male_25.wav'
plot_wave(file)

In [5]:
#Source: https://www.kaggle.com/himanshu007121/audio-analysis-and-augmentation-demo/

def plot_fft(file):
    demo_audio = file #shorter clip
    demo_clip, demo_sample_rate = librosa.load(demo_audio, sr=None)
    demo_xf, demo_vals = custom_fft(demo_clip, demo_sample_rate)

    trace = [go.Scatter(
        x=demo_xf, 
        y=demo_vals,
        line_color='deeppink'
    )]
    layout = go.Layout(
        title = 'Fast Fourier Transform (FFT) <br><sup>Interactive</sup>',
        yaxis = dict(title='Magnitude'),
        xaxis = dict(title='Frequency'),
        )
    fig = go.Figure(data=trace, layout=layout)
    fig.show()

file = r'../input/covid-cough-wavs/cleaned_data/Positive/1084_Positive_male_25.wav'
plot_fft(file)

In [6]:
def fft_new(file):
    fs_rate, signal = wavfile.read(file)
    #print ("Frequency sampling", fs_rate)
    l_audio = len(signal.shape)
    #print ("Channels", l_audio)
    if l_audio == 2:
        signal = signal.sum(axis=1) / 2
    #N = signal.shape[0]
    N = 20000
    #print ("Complete Samplings N", N)
    secs = N / float(fs_rate)
    #print ("secs", secs)
    Ts = 1.0/fs_rate # sampling interval in time
    #print ("Timestep between samples Ts", Ts)
    t = scipy.arange(0, secs, Ts) # time vector as scipy arange field / numpy.ndarray
    FFT = abs(fft(signal))
    FFT_side = FFT[range(N//2)] # one side FFT range
    freqs = scipy.fftpack.fftfreq(signal.size, t[1]-t[0])
    fft_freqs = np.array(freqs)
    freqs_side = freqs[range(N//2)] # one side frequency range
    fft_freqs_side = np.array(freqs_side)
#     p = plt.plot(freqs_side, abs(FFT_side), "b") # plotting the positive fft spectrum
#     plt.xlabel('Frequency (Hz)')
#     plt.ylabel('Magnitude')
#     plt.show()
    mag = abs(FFT_side)
    freq = freqs_side
    return freq, mag

freq, mag = fft_new(file)

In [7]:
def createSeries (series_list):
    series_list = pd.Series(series_list)  
    return series_list

def freq_amp(freq, mag):
    frequencies = createSeries(freq) 
    Amplitude = createSeries(mag)
    data = {"frequencies": frequencies,
            "Amplitude": Amplitude}
    df1 = pd.concat(data, axis = 1)
    df2 = df1[df1['frequencies']<1000]
    return df2

In [8]:
df2 = freq_amp(freq, mag)

In [9]:
df2.head()

In [10]:
def raw_to_agg(df2):
    squares1 = []
    for i in range(51):
        squares1.append(i * 20)

    cols = []
    string = 'string'
    for i in range(1000):
        if i % 20 == 0:
           string ='(' + str(i) + ',' + str(i+20) + ']'
           #Append in a list
           cols.append(string)

    a = pd.cut(df2.frequencies,bins = squares1, labels = cols, ordered=False)
    # a contains the respective bins for all frequency values in data
    return a

In [11]:
a = raw_to_agg(df2)

In [12]:
def percentile(n):
    def percentile_(x):
        return np.percentile(x, n)
    percentile_.__name__ = 'percentile_%s' % n
    return percentile_

In [13]:
def a_to_amp(a, df2):
    freq = a
    amp = df2.Amplitude
    data = {"freq": freq, "amp": amp}
    df3 = pd.concat(data, axis = 1)
    final_data_single_rec = df3.groupby(['freq']).agg({'amp': [percentile(80)]})
    final_data_single_rec.columns = ['_'.join(col) for col in final_data_single_rec.columns.values]
    mn = final_data_single_rec.to_string(index=False, header= False)
    my_list = mn.split("\n")
    a1 = my_list
    b =  [float(float(s)) for s in a1]
    amp = b
    return amp

In [14]:
amp = a_to_amp(a, df2)

In [15]:
type(amp[1])

In [16]:
def gen_add(amp, file):
    gender = file[-13:-7]
    gen = 0
    if gender == 'e_male':
        gen = 1
    gen = float(gen)
    amp.append(gender)
    return amp


In [17]:
#amp = gen_add(amp, file)
print(amp)

In [18]:
# def Convert(lst):
#     res_dct = {lst[i]: lst[i + 1] for i in range(0, len(lst), 2)}
#     return res_dct

# def final_function(file, data):
#     freq, mag = fft_new(file)
#     df2 = freq_amp(freq, mag)
#     amp = raw_to_agg(df2)
#     df_length = len(data)
#     copy = Convert(amp)
#     data.loc[df_length]=pd.DataFrame(copy, index=[df_length]).iloc[df_length]
#     #data.loc[df_length, 'gender'] = file[-13:-7]
#     #data.loc[df_length] = amp
#     return data

In [56]:
def final_function(file, data):
    freq, mag = fft_new(file)
    df2 = freq_amp(freq, mag)
    amp = raw_to_agg(df2)
    amp = a_to_amp(amp, df2)
    #print(amp)
    amp = gen_add(amp, file)
    df_length = len(data)
    #data.loc[df_length]=pd.DataFrame(amp).iloc[df_length]
    #print(df_length)
    data.loc[df_length] = amp
    return data

In [57]:
# def final_function(file, data):
#     freq, mag = fft_new(file)
#     df2 = freq_amp(freq, mag)
#     amp = raw_to_agg(df2)
#     to_append = amp
#     gender = file[-13:-7]
#     # gen = 0
#     # if gender == 'e_male':
#     #     gen = 1
#     to_append.append(gender)
#     df_length = len(data)
#     data.loc[df_length] = to_append
#     return data

In [94]:
cols = []
string = 'string'
for i in range(1000):
    if i % 20 == 0:
        string ='(' + str(i) + ',' + str(i+20) + ']'
        #Append in a list
        cols.append(string)

#Create Dataframe for appending df['Amplitude']
final_data = pd.DataFrame(columns=cols)
final_data_length = len(final_data)
#final_data.loc[final_data_length] = amp --> ISSUE --> RESOLVED

final_data["gender"] = ""
final_data

In [59]:
#type(final_data.loc[len(final_data), final_data.columns != 'gender'])
#final_data.loc[len(final_data)] = amp

In [60]:
file = r'../input/covid-cough-wavs/cleaned_data/Positive/1084_Positive_male_25.wav'
final_data = final_function(file, final_data)

In [61]:
final_data

In [62]:
len(final_data)

In [63]:
gender = file[-13:-7]
print(gender)

In [95]:
# import required module
import os
# assign directory
directory = '../input/covid-cough-wavs/cleaned_data/Negative'
 
# iterate over files in
# that directory
i = 0
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    i += 1
    # checking if it is a file
    if os.path.isfile(f):
        final_function(f, final_data)
        #print(i)

In [96]:
non_cov_data = final_data

In [89]:
cov_data

In [99]:
cov_data.info()

In [100]:
# for i in cov_data.columns:
#     cov_data[i].fillna(int(cov_data[i].mean()), inplace=True)

# for i in non_cov_data.columns:
#     non_cov_data[i].fillna(int(non_cov_data[i].mean()), inplace=True)

In [71]:
non_cov_data

In [97]:
non_cov_data.to_csv('non_cov_data.csv',index=False)

In [98]:
non_cov_data.info()

In [101]:
cov_plt = cov_data
non_cov_plt = non_cov_data

In [102]:
cov_plt['type'] = 'COVID'
non_cov_plt['type'] = 'NON_COVID'

In [107]:
bigdata = pd.concat([cov_plt, non_cov_plt], ignore_index=True, sort=False)

In [108]:
bigdata

In [109]:
bigdata["gender"].replace({"e_male": "male"}, inplace=True)

In [110]:
bigdata

In [112]:
bigdata[cols]